

# **Time Series Data Visualizer (TSDV)**


## TSDV is a tool for Interactive Visualization of High Dimensional Time Series data.
### Till now no such tool is developed that can be used for Visualizing High Dimensional Time Series Data. TSDV is an attempt for doing the same.

### It allows the user to choose from any of the following dimensionality reduction techniques for visualizing their data.

### 1. t-SNE
### 2. PCA
### 3. UMAP
### 4. Open t-SNE
### 5. Joint t-SNE

### This tool allows the user to visualize their data at different timestamps using a time slider. It is helpful because now the user is able to understand the relationship between different clusters generated at different timestamps. It also shows the Cluster Centers for better visualization.
### It also provides the user with an option to select a feature from the dropdown button, so that the whole plot shows the gradient overlay of that selected feature.
### It shows DR quality metrics like Shephard Diagram, Trustworthiness, Continuity and Normalized Stress. So that the user can get an idea about the quality of the resulting plot.

### Installing necessary Libraries

In [ ]:
!pip install 'umap-learn==0.3.10'
!pip install opentsne
!pip install six
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached jedi-0.18.2-py2.py3-none-any.whl (1.6 MB)
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


### Importing necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap

import torch
import os

from sklearn.manifold import TSNE

from openTSNE import TSNEEmbedding
from openTSNE import affinity
from openTSNE import initialization

from ipywidgets import interact, interactive, fixed, interact_manual, GridBox, Layout, VBox
import ipywidgets as widgets

from abc import ABC, abstractmethod
from time import perf_counter
from functools import wraps
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import trustworthiness

from __future__ import division
import warnings
from time import time
from scipy import linalg
import scipy.sparse as sp
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.sparse import csr_matrix
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator
from sklearn.utils import check_array
from sklearn.utils import check_random_state
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.manifold import _utils
from sklearn.manifold import _barnes_hut_tsne
from six import string_types
from sklearn.utils import deprecated
MACHINE_EPSILON = np.finfo(np.double).eps

warnings.filterwarnings('ignore')

scaler = StandardScaler()
pca = PCA(n_components=2)
tsne = TSNE(n_components=2, verbose=1, random_state=123)
map = umap.UMAP(n_components=2)
km = KMeans(n_clusters=10, init='random', n_init=10, max_iter=10000, tol=1e-04, random_state=0)

## **Exploring and preprocessing the Time Series dataset**

### Here we used the Time Series Network dataset.

In [ ]:
df=pd.read_parquet('AGG_GI_NE_NAME_5_MIN-DEST_NE_NAME-2022-10-04_00_00_00_2022-10-04_01_00_00.parquet')
df=df.sample(20000,replace=True)
df['min'] = df['ts'].dt.minute
df.shape

(20000, 64)

In [ ]:
df=df[['DNS Failure Latency (msec)',
       'DNS Success Latency (msec)', 'Internet Latency (msec)',
       'Round Trip Time (msec)', 'HTTP Latency (msec)',
       'DL Data Volume (MB)', 'UL Data Volume (MB)',
       'DL Throughput (kbps)',
       'UL Throughput (kbps)','ts','DEST_NE_NAME','min']]

df_new = df
r = df[['ts','DEST_NE_NAME','min']]
s = scaler.fit_transform(df.drop(['ts','DEST_NE_NAME','min'],axis=1))
t = pd.DataFrame(s,columns=['DNS Failure Latency (msec)',
       'DNS Success Latency (msec)', 'Internet Latency (msec)',
       'Round Trip Time (msec)', 'HTTP Latency (msec)',
       'DL Data Volume (MB)', 'UL Data Volume (MB)',
       'DL Throughput (kbps)',
       'UL Throughput (kbps)'])
df = pd.concat([t.reset_index(drop=True),r.reset_index(drop=True)],axis=1)

# **t-distributed Stochastic Neighbour Embedding (t-SNE)**

### t-Distributed Stochastic Neighbor Embedding (t-SNE) is a technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It is a nonlinear dimensionality reduction technique well-suited for embedding high-dimensional data for visualization in a low-dimensional space of two or three dimensions. Specifically, it models each high-dimensional object by a two- or three-dimensional point in such a way that similar objects are modeled by nearby points and dissimilar objects are modeled by distant points with high probability.

In [ ]:
x_train = df[df["min"]==0]
z_train = df_new[df_new["min"]==0]
x_train = x_train.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_train = x_train.to_numpy()

result = tsne.fit_transform(x_train)

dff0 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff0 = pd.concat([dff0.reset_index(drop=True),z_train.reset_index(drop=True)],axis=1)

x_test = df[df["min"]==5]
z_test = df_new[df_new["min"]==5]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff1 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff1 = pd.concat([dff1.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==10]
z_test = df_new[df_new["min"]==10]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff2 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff2 = pd.concat([dff2.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==15]
z_test = df_new[df_new["min"]==15]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff3 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff3 = pd.concat([dff3.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==20]
z_test = df_new[df_new["min"]==20]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff4 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff4 = pd.concat([dff4.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==25]
z_test = df_new[df_new["min"]==25]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff5 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff5 = pd.concat([dff5.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==30]
z_test = df_new[df_new["min"]==30]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff6 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff6 = pd.concat([dff6.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==35]
z_test = df_new[df_new["min"]==35]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff7 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff7 = pd.concat([dff7.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==40]
z_test = df_new[df_new["min"]==40]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff8 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff8 = pd.concat([dff8.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==45]
z_test = df_new[df_new["min"]==45]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff9 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff9 = pd.concat([dff9.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==50]
z_test = df_new[df_new["min"]==50]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff10 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff10 = pd.concat([dff10.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==55]
z_test = df_new[df_new["min"]==55]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = tsne.fit_transform(x_test)

dff11 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff11 = pd.concat([dff11.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


df_final1 = pd.concat([dff0, dff1, dff2, dff3, dff4, dff5, dff6, dff7, dff8, dff9, dff10, dff11],axis=0)

df_final1.sort_values(by="min",ascending=True,inplace=True)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1666 samples in 0.003s...
[t-SNE] Computed neighbors for 1666 samples in 0.076s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1666
[t-SNE] Computed conditional probabilities for sample 1666 / 1666
[t-SNE] Mean sigma: 0.000040
[t-SNE] KL divergence after 250 iterations with early exaggeration: 52.225365
[t-SNE] KL divergence after 1000 iterations: 0.307482
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1645 samples in 0.003s...
[t-SNE] Computed neighbors for 1645 samples in 0.060s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1645
[t-SNE] Computed conditional probabilities for sample 1645 / 1645
[t-SNE] Mean sigma: 0.000042
[t-SNE] KL divergence after 250 iterations with early exaggeration: 53.038078
[t-SNE] KL divergence after 1000 iterations: 0.323880
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1623 samples in 0.001s...
[t-SNE] Computed neighbors for 1623 samples in 0

# **Principal component analysis (PCA)**

### Principal Component Analysis is an unsupervised learning algorithm that is used for the dimensionality reduction in machine learning. It is a statistical process that converts the observations of correlated features into a set of linearly uncorrelated features with the help of orthogonal transformation. These new transformed features are called the Principal Components. It is one of the popular tools that is used for exploratory data analysis and predictive modeling. It is a technique to draw strong patterns from the given dataset by reducing the variances.

In [ ]:
x_train = df[df["min"]==0]
z_train = df_new[df_new["min"]==0]
x_train = x_train.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_train = x_train.to_numpy()

result = pca.fit_transform(x_train)

dff0 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff0 = pd.concat([dff0.reset_index(drop=True),z_train.reset_index(drop=True)],axis=1)

x_test = df[df["min"]==5]
z_test = df_new[df_new["min"]==5]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff1 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff1 = pd.concat([dff1.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==10]
z_test = df_new[df_new["min"]==10]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff2 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff2 = pd.concat([dff2.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==15]
z_test = df_new[df_new["min"]==15]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff3 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff3 = pd.concat([dff3.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==20]
z_test = df_new[df_new["min"]==20]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff4 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff4 = pd.concat([dff4.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==25]
z_test = df_new[df_new["min"]==25]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff5 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff5 = pd.concat([dff5.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==30]
z_test = df_new[df_new["min"]==30]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff6 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff6 = pd.concat([dff6.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==35]
z_test = df_new[df_new["min"]==35]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff7 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff7 = pd.concat([dff7.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==40]
z_test = df_new[df_new["min"]==40]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff8 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff8 = pd.concat([dff8.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==45]
z_test = df_new[df_new["min"]==45]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff9 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff9 = pd.concat([dff9.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==50]
z_test = df_new[df_new["min"]==50]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff10 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff10 = pd.concat([dff10.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==55]
z_test = df_new[df_new["min"]==55]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = pca.fit_transform(x_test)

dff11 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff11 = pd.concat([dff11.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


df_final4 = pd.concat([dff0, dff1, dff2, dff3, dff4, dff5, dff6, dff7, dff8, dff9, dff10, dff11],axis=0)

df_final4.sort_values(by="min",ascending=True,inplace=True)

# **Uniform Manifold Approximation and Projection (UMAP)**

### Uniform Manifold Approximation and Projection (UMAP) is a dimension reduction technique that can be used for visualisation similarly to t-SNE, but also for general non-linear dimension reduction. The algorithm is founded on three assumptions about the data
###1) The data is uniformly distributed on Riemannian manifold
###2) The Riemannian metric is locally constant (or can be approximated as such)
###3) The manifold is locally connected.

In [ ]:
x_train = df[df["min"]==0]
z_train = df_new[df_new["min"]==0]
x_train = x_train.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_train = x_train.to_numpy()

result = map.fit_transform(x_train)

dff0 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff0 = pd.concat([dff0.reset_index(drop=True),z_train.reset_index(drop=True)],axis=1)

x_test = df[df["min"]==5]
z_test = df_new[df_new["min"]==5]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff1 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff1 = pd.concat([dff1.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==10]
z_test = df_new[df_new["min"]==10]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff2 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff2 = pd.concat([dff2.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==15]
z_test = df_new[df_new["min"]==15]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff3 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff3 = pd.concat([dff3.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==20]
z_test = df_new[df_new["min"]==20]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff4 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff4 = pd.concat([dff4.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==25]
z_test = df_new[df_new["min"]==25]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff5 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff5 = pd.concat([dff5.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==30]
z_test = df_new[df_new["min"]==30]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff6 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff6 = pd.concat([dff6.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==35]
z_test = df_new[df_new["min"]==35]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff7 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff7 = pd.concat([dff7.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==40]
z_test = df_new[df_new["min"]==40]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff8 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff8 = pd.concat([dff8.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==45]
z_test = df_new[df_new["min"]==45]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff9 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff9 = pd.concat([dff9.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==50]
z_test = df_new[df_new["min"]==50]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff10 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff10 = pd.concat([dff10.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==55]
z_test = df_new[df_new["min"]==55]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

result = map.fit_transform(x_test)

dff11 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff11 = pd.concat([dff11.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


df_final5 = pd.concat([dff0, dff1, dff2, dff3, dff4, dff5, dff6, dff7, dff8, dff9, dff10, dff11],axis=0)

df_final5.sort_values(by="min",ascending=True,inplace=True)

# **Open t-SNE**

### OpenTSNE is a modular Python implementation of t-Distributed Stochastic Neighbour Embedding (t-SNE), a popular dimensionality-reduction algorithm for visualising high-dimensional data sets. openTSNE incorporates the latest improvements to the t-SNE algorithm, including the ability to add new data points to existing embeddings .

In [ ]:
x = df[df["min"]==0]
z = df_new[df_new["min"]==0]
x = x.drop(['ts','DEST_NE_NAME','min'],axis=1)
x = x.to_numpy()

affinities_train = affinity.PerplexityBasedNN(
    x,
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=0,
    verbose=True,
)

init_train = initialization.pca(x, random_state=0)

embedding_train = TSNEEmbedding(
    init_train,
    affinities_train,
    negative_gradient_method="fft",
    n_jobs=8,
    verbose=True,
)

embedding_train_1 = embedding_train.optimize(n_iter=400, exaggeration=12, momentum=0.4)

embedding_train_2 = embedding_train_1.optimize(n_iter=800, momentum=0.7)

embedding_train_2 = embedding_train_2.optimize(n_iter=300, momentum=0.5)

dff0 = pd.DataFrame(embedding_train_2, columns = ['comp_1','comp_2'])
dff0 = pd.concat([dff0.reset_index(drop=True), z.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==5]
z_test = df_new[df_new["min"]==5]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff1 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff1 = pd.concat([dff1.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==10]
z_test = df_new[df_new["min"]==10]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff2 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff2 = pd.concat([dff2.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==15]
z_test = df_new[df_new["min"]==15]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff3 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff3 = pd.concat([dff3.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==20]
z_test = df_new[df_new["min"]==20]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff4 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff4 = pd.concat([dff4.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==25]
z_test = df_new[df_new["min"]==25]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff5 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff5 = pd.concat([dff5.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==30]
z_test = df_new[df_new["min"]==30]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff6 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff6 = pd.concat([dff6.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==35]
z_test = df_new[df_new["min"]==35]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff7 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff7 = pd.concat([dff7.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==40]
z_test = df_new[df_new["min"]==40]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff8 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff8 = pd.concat([dff8.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==45]
z_test = df_new[df_new["min"]==45]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff9 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff9 = pd.concat([dff9.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==50]
z_test = df_new[df_new["min"]==50]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff10 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff10 = pd.concat([dff10.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==55]
z_test = df_new[df_new["min"]==55]
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.to_numpy()

embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

embedding_test_1 = embedding_test.optimize(n_iter=300, learning_rate=0.1, momentum=0.8)

dff11 = pd.DataFrame(embedding_test_1, columns = ['comp_1','comp_2'])
dff11 = pd.concat([dff11.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


df_final2 = pd.concat([dff0, dff1, dff2, dff3, dff4, dff5, dff6, dff7, dff8, dff9, dff10, dff11],axis=0)

df_final2.sort_values(by="min",ascending=True,inplace=True)

===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.59 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.06 seconds
===> Running optimization with exaggeration=12.00, lr=138.83 for 400 iterations...
Iteration   50, KL divergence 3.6252, 50 iterations in 0.8347 sec
Iteration  100, KL divergence 2.2279, 50 iterations in 0.8986 sec
Iteration  150, KL divergence 1.9523, 50 iterations in 0.7203 sec
Iteration  200, KL divergence 1.8361, 50 iterations in 0.5434 sec
Iteration  250, KL divergence 1.7737, 50 iterations in 0.5202 sec
Iteration  300, KL divergence 1.7358, 50 iterations in 0.5314 sec
Iteration  350, KL divergence 1.7107, 50 iterations in 0.5258 sec
Iteration  400, KL divergence 1.6926, 50 iterations in 0.5446 sec
   --> Time elapsed: 5.12 seconds
===> Running optimization with exaggeration=1.00, lr=1666.00 for 800 iterations...
Iteration   50, KL divergence 0.4654, 50 iterations in 0.5621 sec
Iteration

# **Implementation of Joint t-Stochastic Neighbour Embedding (Joint t-SNE)**

### Joint t-Stochastic Neighbour Embedding (Joint t-SNE) is a technique to generate comparable projections of multiple high-dimensional datasets. When a series of high-dimensional datasets, such as datasets changing over time, is projected independently using t-SNE, misaligned layouts are obtained. Even items with identical features across datasets are projected to different locations, making the technique unsuitable for comparison tasks. To tackle this problem, we use joint t-SNE that in turn uses edge similarity, which captures the similarities between two adjacent time frames based on the Graphlet Frequency Distribution (GFD). It then integrates a novel loss term into the t-SNE loss function to preserve the vectors between projected points across the projections, allowing these points to serve as visual landmarks for direct comparisons between projections.

In [ ]:
def recall(Fvec, label):
    N = len(label)
    print(N)
    disList = []

    D = Fvec.mm(torch.t(Fvec))

    D[torch.eye(N).byte()] = -1

    sort_D, indices = torch.sort(D,1,descending=True)
    print(indices[0])
    for i in range(0,N):
        dis = []
        distance = sort_D[i]
        index = indices[i]

        for j in range(0,N):
            if label[index[j].item()]==label[i]:
                dis.append(distance[j].item())
                break;

        for k in range(0,N):
            if label[index[k].item()]!=label[i]:
                dis.append(distance[k].item())
                break;

        disList.append(dis)
    return disList

def draw_sca(disList,name='',savepath='./'):

    disList = np.asarray(disList)
    print(disList.shape)
    A=disList[:,0]
    B=disList[:,1]
    diff=B>A
    same=A>B
    plt.figure(figsize=(6.4,6.4))
    plt.title(name)
    plt.xlim(0.3,1.0)
    plt.ylim(0.3,1.0)
    plt.xlabel('Similarity (Same class)',fontsize=22)
    plt.ylabel('Similarity (Different class)',fontsize=22)
    samex = A[same]
    samey = B[same]
    difx = A[diff]
    dify = B[diff]
    plt.scatter(samex,samey,c='c',s=0.1)
    plt.scatter(difx,dify,color=[1.0, 0.5, 0.25],s=0.1)
    if not os.path.isdir(savepath):
        os.makedirs(savepath)
    plt.savefig(savepath+'scatter_plot.jpg')

def gene_sca(vecPath,labelPath,name='',savepath='./'):
    Fvec = np.load(vecPath)
    label = np.load(labelPath)
    Fvec = torch.Tensor(Fvec)
    disList = recall(Fvec,label)
    draw_sca(disList,name,savepath)

In [ ]:
def _joint_probabilities(distances, desired_perplexity, verbose):
    distances = distances.astype(np.float32, copy=False)
    conditional_P = _utils._binary_search_perplexity(
        distances, None, desired_perplexity, verbose)
    P = conditional_P + conditional_P.T
    sum_P = np.maximum(np.sum(P), MACHINE_EPSILON)
    P = np.maximum(squareform(P) / sum_P, MACHINE_EPSILON)
    return P

def _joint_probabilities_nn(distances, neighbors,desired_perplexity, verbose):
    t0 = time()
    n_samples, k = neighbors.shape
    distances = distances.astype(np.float32, copy=False)
    neighbors = neighbors.astype(np.int64, copy=False)
    conditional_P = _utils._binary_search_perplexity(
        distances, desired_perplexity, verbose)
    assert np.all(np.isfinite(conditional_P)), \
        "All probabilities should be finite"

    P = csr_matrix((conditional_P.ravel(), neighbors.ravel(),
                    range(0, n_samples * k + 1, k)),
                   shape=(n_samples, n_samples))
    P = P + P.T

    sum_P = np.maximum(P.sum(), MACHINE_EPSILON)
    P /= sum_P

    assert np.all(np.abs(P.data) <= 1.0)
    if verbose >= 2:
        duration = time() - t0
        print("[t-SNE] Computed conditional probabilities in {:.3f}s"
              .format(duration))
    return P

def _kl_divergence(params, P, degrees_of_freedom, n_samples, n_components,
                   skip_num_points=0, compute_error=True):
    X_embedded = params.reshape(n_samples, n_components)

    dist = pdist(X_embedded, "sqeuclidean")
    dist /= degrees_of_freedom
    dist += 1.
    dist **= (degrees_of_freedom + 1.0) / -2.0
    Q = np.maximum(dist / (2.0 * np.sum(dist)), MACHINE_EPSILON)

    if compute_error:
        kl_divergence = 2.0 * np.dot(
            P, np.log(np.maximum(P, MACHINE_EPSILON) / Q))
    else:
        kl_divergence = np.nan

    grad = np.ndarray((n_samples, n_components), dtype=params.dtype)
    PQd = squareform((P - Q) * dist)
    for i in range(skip_num_points, n_samples):
        grad[i] = np.dot(np.ravel(PQd[i], order='K'),
                         X_embedded[i] - X_embedded)
    grad = grad.ravel()
    c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom
    grad *= c

    return kl_divergence, grad

def _kl_divergence_bh(params, P, degrees_of_freedom, n_samples, n_components,
                      angle=0.5, skip_num_points=0, verbose=False,
                      compute_error=True):
    params = params.astype(np.float32, copy=False)
    X_embedded = params.reshape(n_samples, n_components)

    val_P = P.data.astype(np.float32, copy=False)
    neighbors = P.indices.astype(np.int64, copy=False)
    indptr = P.indptr.astype(np.int64, copy=False)

    grad = np.zeros(X_embedded.shape, dtype=np.float32)
    error = _barnes_hut_tsne.gradient(val_P, X_embedded, neighbors, indptr,
                                      grad, angle, n_components, verbose,
                                      dof=degrees_of_freedom,
                                      compute_error=compute_error)
    c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom
    grad = grad.ravel()
    grad *= c

    return error, grad

def _kl_divergence_yoke(params_X,params_Y, P_X,P_Y, degrees_of_freedom, n_samples, n_components,alpha=0,fixed_Y=False,y=None,
                   skip_num_points=0, compute_error=True,oneplot=False):

    X_embedded = params_X.reshape(n_samples, n_components)
    Y_embedded = params_Y.reshape(n_samples, n_components)

    dist_X = pdist(X_embedded, "sqeuclidean")
    dist_X /= degrees_of_freedom
    dist_X += 1.
    dist_X **= (degrees_of_freedom + 1.0) / -2.0
    Q_X = np.maximum(dist_X / (2.0 * np.sum(dist_X)), MACHINE_EPSILON)

    if not fixed_Y:
        dist_Y = pdist(Y_embedded, "sqeuclidean")
        dist_Y /= degrees_of_freedom
        dist_Y += 1.
        dist_Y **= (degrees_of_freedom + 1.0) / -2.0
        Q_Y = np.maximum(dist_Y / (2.0 * np.sum(dist_Y)), MACHINE_EPSILON)
    if compute_error:
        kl_divergence_X = 2.0 * np.dot(
            P_X, np.log(np.maximum(P_X, MACHINE_EPSILON) / Q_X))
        if not fixed_Y:
            kl_divergence_Y = 2.0 * np.dot(
                P_Y, np.log(np.maximum(P_Y, MACHINE_EPSILON) / Q_Y))
    else:
        kl_divergence_X = np.nan
        kl_divergence_Y = np.nan

    grad_X = np.zeros(X_embedded.shape, dtype=np.float32)
    grad_Y = np.zeros(Y_embedded.shape, dtype=np.float32)
    PQd_X = squareform((P_X - Q_X) * dist_X)
    if not fixed_Y:
        PQd_Y = squareform((P_Y - Q_Y) * dist_Y)
        for i in range(0, n_samples):
            grad_Y[i] = np.dot(np.ravel(PQd_Y[i], order='K'),
                            Y_embedded[i] - Y_embedded)
    for i in range(0, n_samples):
        grad_X[i] = np.dot(np.ravel(PQd_X[i], order='K'),
                        X_embedded[i] - X_embedded)

    if y is None:
        grad_X += 2*alpha* (X_embedded-Y_embedded)
        if not fixed_Y:
            grad_Y += 2*alpha* (Y_embedded-X_embedded)

    if y is not None:
        cluster_grad_X = np.zeros(grad_X.shape)
        cluster_grad_Y = np.zeros(grad_Y.shape)
        dst = {}
        gradient = {}
        for i in set(y):
            index = np.where(y==i)
            Xcenter = X_embedded[index].sum(axis=0)/len(index[0])
            Ycenter = Y_embedded[index].sum(axis=0)/len(index[0])
            dst[i]=(distance.euclidean(Xcenter,Ycenter))
            gradient[i]=(Xcenter-Ycenter)/len(index[0])
        threshold = np.percentile(np.asarray(list(dst.values())),ratio*100)
        for key in dst.keys():
            if dst[key]>threshold:
                dst[key]=0
                gradient[key]=0
        for i in set(y):
            index = np.where(y==i)
            cluster_grad_X[index] = gradient[i]
            if not fixed_Y:
                cluster_grad_Y[index] = -gradient[i]
        grad_X = grad_X + 2*alpha*cluster_grad_X
        grad_Y = grad_Y + 2*alpha*cluster_grad_Y
    grad_X = grad_X.ravel()
    grad_Y = grad_Y.ravel()
    c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom
    grad_X *= c
    grad_Y *= c

    error = kl_divergence_X+kl_divergence_Y+alpha*np.power((X_embedded-Y_embedded),2).sum()

    return error, errorx, errory, grad_X, grad_Y


def _kl_divergence_yoke_bh(params_X,params_Y, P_X,P_Y, degrees_of_freedom, n_samples, n_components,alpha=0,
                      angle=0.5, skip_num_points=0, verbose=False,fixed_Y=False,y=None,
                      compute_error=True,oneplot=False,ratio=1.0):

    params_X = params_X.astype(np.float32, copy=False)
    params_Y = params_Y.astype(np.float32, copy=False)
    X_embedded = params_X.reshape(n_samples, n_components)
    Y_embedded = params_Y.reshape(n_samples, n_components)

    val_P_X = P_X.data.astype(np.float32, copy=False)
    neighbors_X = P_X.indices.astype(np.int64, copy=False)
    indptr_X = P_X.indptr.astype(np.int64, copy=False)

    if not fixed_Y:
        val_P_Y = P_Y.data.astype(np.float32, copy=False)
        neighbors_Y = P_Y.indices.astype(np.int64, copy=False)
        indptr_Y = P_Y.indptr.astype(np.int64, copy=False)



    grad_X = np.zeros(X_embedded.shape, dtype=np.float32)

    grad_Y = np.zeros(Y_embedded.shape, dtype=np.float32)


    errorx = _barnes_hut_tsne.gradient(val_P_X, X_embedded, neighbors_X, indptr_X,
                                      grad_X, angle, n_components, verbose,
                                      dof=degrees_of_freedom,
                                      compute_error=compute_error)


    errory = 0

    if not fixed_Y and not oneplot:
        errory = _barnes_hut_tsne.gradient(val_P_Y, Y_embedded, neighbors_Y, indptr_Y,
                                      grad_Y, angle, n_components, verbose,
                                      dof=degrees_of_freedom,
                                      compute_error=compute_error)
    if oneplot and not fixed_Y:
        errory = _barnes_hut_tsne.gradient(val_P_Y, X_embedded, neighbors_Y, indptr_Y,
                                      grad_Y, angle, n_components, verbose,
                                      dof=degrees_of_freedom,
                                      compute_error=compute_error)
        grad_X += grad_Y
    c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom

    if not oneplot:
        if y is None:
            grad_X += 2*alpha* (X_embedded-Y_embedded)

            if not fixed_Y:
                grad_Y += 2*alpha* (Y_embedded-X_embedded)

        if y is not None:
            cluster_grad_X = np.zeros(grad_X.shape)
            cluster_grad_Y = np.zeros(grad_Y.shape)
            dst = {}
            gradient = {}
            for i in set(y):
                index = np.where(y==i)
                Xcenter = X_embedded[index].sum(axis=0)/len(index[0])
                Ycenter = Y_embedded[index].sum(axis=0)/len(index[0])
                dst[i]=(distance.euclidean(Xcenter,Ycenter))
                gradient[i]=(Xcenter-Ycenter)/len(index[0])
            threshold = np.percentile(np.asarray(list(dst.values())),ratio*100)
            for key in dst.keys():
                if dst[key]>threshold:
                    dst[key]=0
                    gradient[key]=0
            for i in set(y):
                index = np.where(y==i)
                cluster_grad_X[index] = gradient[i]
                if not fixed_Y:
                    cluster_grad_Y[index] = -gradient[i]
            grad_X = grad_X + 2*alpha*cluster_grad_X
            grad_Y = grad_Y + 2*alpha*cluster_grad_Y

    grad_X = grad_X.ravel()
    grad_X *= c

    grad_Y = grad_Y.ravel()
    grad_Y *= c

    error = errorx+errory+alpha*np.power((X_embedded-Y_embedded),2).sum()

    return error, errorx, errory, grad_X, grad_Y





def _gradient_descent_yoked(objective, X,Y, it, n_iter,
                      n_iter_check=1, n_iter_without_progress=300,
                      momentum=0.8, learning_rate=200.0, min_gain=0.01,
                      min_grad_norm=1e-7, verbose=0, args=None,fixed_Y=False,y=None,alpha=0, kwargs=None):

    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}

    px = X.copy().ravel()
    py = Y.copy().ravel()
    updatex = np.zeros_like(px)
    gainsx = np.ones_like(px)
    updatey = np.zeros_like(py)
    gainsy = np.ones_like(py)

    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = i = it

    tic = time()
    for i in range(it, n_iter):
        check_convergence = (i + 1) % n_iter_check == 0
        kwargs['compute_error'] = check_convergence or i == n_iter - 1

        error, errorx, errory, gradx, grady = objective(px,py, *args, **kwargs)
        gradx_norm = linalg.norm(gradx)
        grady_norm = linalg.norm(grady)


        incx = updatex * gradx < 0.0
        decx = np.invert(incx)
        gainsx[incx] += 0.2
        gainsx[decx] *= 0.8

        incy = updatey * grady < 0.0
        decy = np.invert(incy)
        gainsy[incy] += 0.2
        gainsy[decy] *= 0.8


        np.clip(gainsx, min_gain, np.inf, out=gainsx)
        gradx *= gainsx
        updatex = momentum * updatex - learning_rate * gradx
        px += updatex

        np.clip(gainsy, min_gain, np.inf, out=gainsy)
        grady *= gainsy
        updatey = momentum * updatey - learning_rate * grady
        py += updatey

        if check_convergence:
            toc = time()
            duration = toc - tic
            tic = toc

            if verbose >= 2:
                print("[t-SNE] Iteration %d: error = %.7f,"
                      " gradient norm = %.7f"
                      " (%s iterations in %0.3fs)"
                      % (i + 1, error, grad_norm, n_iter_check, duration))

            if error < best_error:
                best_error = error
                best_iter = i
            elif i - best_iter > n_iter_without_progress:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: did not make any progress "
                          "during the last %d episodes. Finished."
                          % (i + 1, n_iter_without_progress))
                break
            if gradx_norm <= min_grad_norm and grady_norm <= min_grad_norm:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: gradient norm %f. Finished."
                          % (i + 1, grad_norm))
                break

    return px,py, error,errorx,errory, i


def _gradient_descent(objective, p0, it, n_iter,
                      n_iter_check=1, n_iter_without_progress=300,
                      momentum=0.8, learning_rate=200.0, min_gain=0.01,
                      min_grad_norm=1e-7, verbose=0, args=None, kwargs=None):
    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}

    p = p0.copy().ravel()
    update = np.zeros_like(p)
    gains = np.ones_like(p)
    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = i = it

    tic = time()
    for i in range(it, n_iter):
        check_convergence = (i + 1) % n_iter_check == 0
        kwargs['compute_error'] = check_convergence or i == n_iter - 1

        error, grad = objective(p, *args, **kwargs)
        grad_norm = linalg.norm(grad)

        inc = update * grad < 0.0
        dec = np.invert(inc)
        gains[inc] += 0.2
        gains[dec] *= 0.8
        np.clip(gains, min_gain, np.inf, out=gains)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update

        if check_convergence:
            toc = time()
            duration = toc - tic
            tic = toc

            if verbose >= 2:
                print("[t-SNE] Iteration %d: error = %.7f,"
                      " gradient norm = %.7f"
                      " (%s iterations in %0.3fs)"
                      % (i + 1, error, grad_norm, n_iter_check, duration))

            if error < best_error:
                best_error = error
                best_iter = i
            elif i - best_iter > n_iter_without_progress:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: did not make any progress "
                          "during the last %d episodes. Finished."
                          % (i + 1, n_iter_without_progress))
                break
            if grad_norm <= min_grad_norm:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: gradient norm %f. Finished."
                          % (i + 1, grad_norm))
                break

    return p, error, i


def trustworthiness(X, X_embedded, n_neighbors=5,
                    precomputed=False, metric='euclidean'):

    if precomputed:
        warnings.warn("The flag 'precomputed' has been deprecated in version "
                      "0.20 and will be removed in 0.22. See 'metric' "
                      "parameter instead.", DeprecationWarning)
        metric = 'precomputed'
    dist_X = pairwise_distances(X, metric=metric)
    ind_X = np.argsort(dist_X, axis=1)
    ind_X_embedded = NearestNeighbors(n_neighbors).fit(X_embedded).kneighbors(
        return_distance=False)

    n_samples = X.shape[0]
    t = 0.0
    ranks = np.zeros(n_neighbors)
    for i in range(n_samples):
        for j in range(n_neighbors):
            ranks[j] = np.where(ind_X[i] == ind_X_embedded[i, j])[0][0]
        ranks -= n_neighbors
        t += np.sum(ranks[ranks > 0])
    t = 1.0 - t * (2.0 / (n_samples * n_neighbors *
                          (2.0 * n_samples - 3.0 * n_neighbors - 1.0)))
    return t


class Yoked_TSNE(BaseEstimator):
    _EXPLORATION_N_ITER = 250

    _N_ITER_CHECK = 50

    def __init__(self, n_components=2, perplexity=30.0,
                 early_exaggeration=12.0, learning_rate=200.0, n_iter=1000,
                 n_iter_without_progress=300, min_grad_norm=1e-7,
                 metric="euclidean",init_ratio=1,init="random", init_X="random", init_Y="random", verbose=0,
                 random_state=None, method='barnes_hut', angle=0.5):
        self.n_components = n_components
        self.perplexity = perplexity
        self.early_exaggeration = early_exaggeration
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.n_iter_without_progress = n_iter_without_progress
        self.min_grad_norm = min_grad_norm
        self.metric = metric
        self.init_ratio=1
        self.init = init
        self.init_X = init_X
        self.init_Y = init_Y
        self.verbose = verbose
        self.random_state = random_state
        self.method = method
        self.angle = angle

    def _fit(self, X, skip_num_points=0):
        if self.method not in ['barnes_hut', 'exact']:
            raise ValueError("'method' must be 'barnes_hut' or 'exact'")
        if self.angle < 0.0 or self.angle > 1.0:
            raise ValueError("'angle' must be between 0.0 - 1.0")
        if self.metric == "precomputed":
            if isinstance(self.init, string_types) and self.init == 'pca':
                raise ValueError("The parameter init=\"pca\" cannot be "
                                 "used with metric=\"precomputed\".")
            if X.shape[0] != X.shape[1]:
                raise ValueError("X should be a square distance matrix")
            if np.any(X < 0):
                raise ValueError("All distances should be positive, the "
                                 "precomputed distances given as X is not "
                                 "correct")
        if self.method == 'barnes_hut' and sp.issparse(X):
            raise TypeError('A sparse matrix was passed, but dense '
                            'data is required for method="barnes_hut". Use '
                            'X.toarray() to convert to a dense numpy array if '
                            'the array is small enough for it to fit in '
                            'memory. Otherwise consider dimensionality '
                            'reduction techniques (e.g. TruncatedSVD)')
        if self.method == 'barnes_hut':
            X = check_array(X, ensure_min_samples=2,
                            dtype=[np.float32, np.float64])
        else:
            X = check_array(X, accept_sparse=['csr', 'csc', 'coo'],
                            dtype=[np.float32, np.float64])
        if self.method == 'barnes_hut' and self.n_components > 3:
            raise ValueError("'n_components' should be inferior to 4 for the "
                             "barnes_hut algorithm as it relies on "
                             "quad-tree or oct-tree.")
        random_state = check_random_state(self.random_state)

        if self.early_exaggeration < 1.0:
            raise ValueError("early_exaggeration must be at least 1, but is {}"
                             .format(self.early_exaggeration))

        if self.n_iter < 250:
            raise ValueError("n_iter should be at least 250")

        n_samples = X.shape[0]

        neighbors_nn = None
        if self.method == "exact":
            if self.metric == "precomputed":
                distances = X
            else:
                if self.verbose:
                    print("[t-SNE] Computing pairwise distances...")

                if self.metric == "euclidean":
                    distances = pairwise_distances(X, metric=self.metric,
                                                   squared=True)
                else:
                    distances = pairwise_distances(X, metric=self.metric)

                if np.any(distances < 0):
                    raise ValueError("All distances should be positive, the "
                                     "metric given is not correct")

            P = _joint_probabilities(distances, self.perplexity, self.verbose)
            assert np.all(np.isfinite(P)), "All probabilities should be finite"
            assert np.all(P >= 0), "All probabilities should be non-negative"
            assert np.all(P <= 1), ("All probabilities should be less "
                                    "or then equal to one")

        else:
            k = min(n_samples - 1, int(3. * self.perplexity + 1))

            if self.verbose:
                print("[t-SNE] Computing {} nearest neighbors...".format(k))

            knn = NearestNeighbors(algorithm='auto', n_neighbors=k,
                                   metric=self.metric)
            t0 = time()
            knn.fit(X)
            duration = time() - t0
            if self.verbose:
                print("[t-SNE] Indexed {} samples in {:.3f}s...".format(
                    n_samples, duration))

            t0 = time()
            distances_nn, neighbors_nn = knn.kneighbors(
                None, n_neighbors=k)
            duration = time() - t0
            if self.verbose:
                print("[t-SNE] Computed neighbors for {} samples in {:.3f}s..."
                      .format(n_samples, duration))

            del knn

            if self.metric == "euclidean":
                distances_nn **= 2

            P = _joint_probabilities_nn(distances_nn, neighbors_nn,
                                        self.perplexity, self.verbose)

        if isinstance(self.init, np.ndarray):
            X_embedded = self.init
        elif self.init == 'pca':
            pca = PCA(n_components=self.n_components, svd_solver='randomized',
                      random_state=random_state)
            X_embedded = pca.fit_transform(X).astype(np.float32, copy=False)
        elif self.init == 'random':
            X_embedded = self.init_ratio*1e-4 * random_state.randn(
                n_samples, self.n_components).astype(np.float32)
        else:
            raise ValueError("'init' must be 'pca', 'random', or "
                             "a numpy array")

        degrees_of_freedom = max(self.n_components - 1, 1)

        return self._tsne(P, degrees_of_freedom, n_samples,
                          X_embedded=X_embedded,
                          neighbors=neighbors_nn,
                          skip_num_points=skip_num_points)


    def _yoke(self, X,Y,alpha,y=None,fixed_Y=False,oneplot=False,skip_num_points=0,ratio=1.0):
        if self.method not in ['barnes_hut', 'exact']:
            raise ValueError("'method' must be 'barnes_hut' or 'exact'")
        if self.angle < 0.0 or self.angle > 1.0:
            raise ValueError("'angle' must be between 0.0 - 1.0")
        if self.metric == "precomputed":
            if isinstance(self.init_X, string_types) and self.init == 'pca':
                raise ValueError("The parameter init=\"pca\" cannot be "
                                 "used with metric=\"precomputed\".")
            if X.shape[0] != X.shape[1]:
                raise ValueError("X should be a square distance matrix")
            if np.any(X < 0):
                raise ValueError("All distances should be positive, the "
                                 "precomputed distances given as X is not "
                                 "correct")
            if isinstance(self.init_Y, string_types) and self.init == 'pca':
                raise ValueError("The parameter init=\"pca\" cannot be "
                                 "used with metric=\"precomputed\".")
            if fixed_Y==False and Y.shape[0] != Y.shape[1]:
                raise ValueError("X should be a square distance matrix")
            if fixed_Y==False and np.any(Y < 0):
                raise ValueError("All distances should be positive, the "
                                 "precomputed distances given as X is not "
                                 "correct")
        if (self.method == 'barnes_hut' and sp.issparse(X)) or (fixed_Y==False and self.method == 'barnes_hut' and sp.issparse(Y)):
            raise TypeError('A sparse matrix was passed, but dense '
                            'data is required for method="barnes_hut". Use '
                            'X.toarray() to convert to a dense numpy array if '
                            'the array is small enough for it to fit in '
                            'memory. Otherwise consider dimensionality '
                            'reduction techniques (e.g. TruncatedSVD)')
        if self.method == 'barnes_hut':
            X = check_array(X, ensure_min_samples=2,
                            dtype=[np.float32, np.float64])
            if not fixed_Y:
                Y = check_array(Y, ensure_min_samples=2,
                            dtype=[np.float32, np.float64])
        else:
            X = check_array(X, accept_sparse=['csr', 'csc', 'coo'],
                            dtype=[np.float32, np.float64])
            if not fixed_Y:
                Y = check_array(Y, accept_sparse=['csr', 'csc', 'coo'],
                            dtype=[np.float32, np.float64])
        if self.method == 'barnes_hut' and self.n_components > 3:
            raise ValueError("'n_components' should be inferior to 4 for the "
                             "barnes_hut algorithm as it relies on "
                             "quad-tree or oct-tree.")
        random_state = check_random_state(self.random_state)

        if self.early_exaggeration < 1.0:
            raise ValueError("early_exaggeration must be at least 1, but is {}"
                             .format(self.early_exaggeration))

        if self.n_iter < 250:
            raise ValueError("n_iter should be at least 250")

        n_samples = X.shape[0]

        neighbors_nn_X = None
        neighbors_nn_Y = None
        if self.method == "exact":
            if self.metric == "precomputed":
                distances_X = X
                distances_Y = Y
            else:
                if self.verbose:
                    print("[t-SNE] Computing pairwise distances...")

                if self.metric == "euclidean":
                    distances_X = pairwise_distances(X, metric=self.metric,
                                                   squared=True)
                    if not fixed_Y:
                        distances_Y = pairwise_distances(Y, metric=self.metric,
                                                   squared=True)
                else:
                    distances_X = pairwise_distances(X, metric=self.metric)
                    if not fixed_Y:
                        distances_Y = pairwise_distances(Y, metric=self.metric)

                if np.any(distances_X < 0):
                    raise ValueError("All distances should be positive, the "
                                     "metric given is not correct")
                if np.any(distances_Y<0):
                    raise ValueError("All distances should be positive, the "
                                     "metric given is not correct")

            P_X = _joint_probabilities(distances_X, self.perplexity, self.verbose)
            assert np.all(np.isfinite(P_X)), "All probabilities should be finite"
            assert np.all(P_X >= 0), "All probabilities should be non-negative"
            assert np.all(P_X <= 1), ("All probabilities should be less "
                                    "or then equal to one")
            if not fixed_Y:
                P_Y = _joint_probabilities(distances_Y, self.perplexity, self.verbose)
                assert np.all(np.isfinite(P_Y)), "All probabilities should be finite"
                assert np.all(P_Y >= 0), "All probabilities should be non-negative"
                assert np.all(P_Y <= 1), ("All probabilities should be less "
                                    "or then equal to one")


        else:
            k = min(n_samples - 1, int(3. * self.perplexity + 1))
            if self.verbose:
                print("[t-SNE] Computing {} nearest neighbors...".format(k))

            knn1 = NearestNeighbors(algorithm='auto', n_neighbors=k,
                                   metric=self.metric)
            t0 = time()
            knn1.fit(X)
            duration = time() - t0
            if self.verbose:
                print("[t-SNE] Indexed {} samples in {:.3f}s...".format(
                    n_samples, duration))

            t0 = time()
            distances_nn_X, neighbors_nn_X = knn1.kneighbors(
                None, n_neighbors=k)
            duration = time() - t0
            if self.verbose:
                print("[t-SNE] Computed neighbors for {} samples in {:.3f}s..."
                      .format(n_samples, duration))

            del knn1

            if not fixed_Y:
                knn2 = NearestNeighbors(algorithm='auto', n_neighbors=k,
                                   metric=self.metric)
                t0 = time()
                knn2.fit(Y)
                duration = time() - t0
                if self.verbose:
                    print("[t-SNE] Indexed {} samples in {:.3f}s...".format(
                        n_samples, duration))

                t0 = time()
                distances_nn_Y, neighbors_nn_Y = knn2.kneighbors(
                    None, n_neighbors=k)
                duration = time() - t0
                if self.verbose:
                    print("[t-SNE] Computed neighbors for {} samples in {:.3f}s..."
                          .format(n_samples, duration))

                del knn2

            if self.metric == "euclidean":
                distances_nn_X **= 2
                if not fixed_Y:
                    distances_nn_Y **= 2

            P_X = _joint_probabilities_nn(distances_nn_X, neighbors_nn_X,
                                        self.perplexity, self.verbose)
            P_Y = 0
            if not fixed_Y:
                P_Y = _joint_probabilities_nn(distances_nn_Y, neighbors_nn_Y,
                                        self.perplexity, self.verbose)

        if isinstance(self.init_Y, np.ndarray):
            Y_embedded = self.init_Y
        elif self.init_Y == 'pca':
            pca = PCA(n_components=self.n_components, svd_solver='randomized',
                      random_state=random_state)
            Y_embedded = pca.fit_transform(X).astype(np.float32, copy=False)
        elif self.init_Y == 'random':
            Y_embedded = 1e-4 * random_state.randn(n_samples, self.n_components).astype(np.float32)
            if not fixed_Y:
                Y_embedded = 1e-4 * random_state.randn(n_samples, self.n_components).astype(np.float32)
        if isinstance(self.init_X,np.ndarray):
            X_embedded = self.init_X
        elif self.init_X == 'pca':
            pca = PCA(n_components=self.n_components, svd_solver='randomized',
                      random_state=random_state)
            X_embedded = pca.fit_transform(X).astype(np.float32, copy=False)
            if not fixed_Y:
                Y_embedded = pca.fit_transform(Y).astype(np.float32, copy=False)
        elif self.init_X == 'random':
            X_embedded = self.init_ratio*1e-4 * random_state.randn(n_samples, self.n_components).astype(np.float32)
            if not fixed_Y:
                Y_embedded = self.init_ratio*1e-4 * random_state.randn(n_samples, self.n_components).astype(np.float32)
        elif not isinstance(self.init_Y, np.ndarray) and fixed_Y==True:
            raise ValueError("Must be given fixed Y")
        else:
            raise ValueError("'init' must be 'pca', 'random', or "
                             "a numpy array")

        if self.init_Y == 'same':
            Y_embedded = X_embedded

        degrees_of_freedom = max(self.n_components - 1, 1)

        return self._yoke_tsne(P_X,P_Y, degrees_of_freedom, n_samples,
                          X_embedded=X_embedded,
                          Y_embedded=Y_embedded,
                          neighbors_X=neighbors_nn_X,
                          neighbors_Y=neighbors_nn_Y,
                          skip_num_points=skip_num_points,y=y,alpha=alpha,fixed_Y=fixed_Y,oneplot=oneplot,ratio=ratio)


    @property
    @deprecated("Attribute n_iter_final was deprecated in version 0.19 and "
                "will be removed in 0.21. Use ``n_iter_`` instead")
    def n_iter_final(self):
        return self.n_iter_

    def _tsne(self, P, degrees_of_freedom, n_samples, X_embedded,
              neighbors=None, skip_num_points=0):
        params = X_embedded.ravel()

        opt_args = {
            "it": 0,
            "n_iter_check": self._N_ITER_CHECK,
            "min_grad_norm": self.min_grad_norm,
            "learning_rate": self.learning_rate,
            "verbose": self.verbose,
            "kwargs": dict(skip_num_points=skip_num_points),
            "args": [P, degrees_of_freedom, n_samples, self.n_components],
            "n_iter_without_progress": self._EXPLORATION_N_ITER,
            "n_iter": self._EXPLORATION_N_ITER,
            "momentum": 0.5,
        }
        if self.method == 'barnes_hut':
            obj_func = _kl_divergence_bh
            opt_args['kwargs']['angle'] = self.angle
            opt_args['kwargs']['verbose'] = self.verbose
        else:
            obj_func = _kl_divergence

        P *= self.early_exaggeration
        params, kl_divergence, it = _gradient_descent(obj_func, params,
                                                      **opt_args)
        if self.verbose:
            print("[t-SNE] KL divergence after %d iterations with early "
                  "exaggeration: %f" % (it + 1, kl_divergence))

        P /= self.early_exaggeration
        remaining = self.n_iter - self._EXPLORATION_N_ITER
        if it < self._EXPLORATION_N_ITER or remaining > 0:
            opt_args['n_iter'] = self.n_iter
            opt_args['it'] = it + 1
            opt_args['momentum'] = 0.8
            opt_args['n_iter_without_progress'] = self.n_iter_without_progress
            params, kl_divergence, it = _gradient_descent(obj_func, params,
                                                          **opt_args)

        self.n_iter_ = it

        if self.verbose:
            print("[t-SNE] KL divergence after %d iterations: %f"
                  % (it + 1, kl_divergence))

        X_embedded = params.reshape(n_samples, self.n_components)
        self.kl_divergence_ = kl_divergence

        return X_embedded


    def _yoke_tsne(self, P_X, P_Y, degrees_of_freedom, n_samples, X_embedded, Y_embedded,
              neighbors_X=None,neighbors_Y=None,y=None,fixed_Y = False, skip_num_points=0,alpha=0,oneplot=False,ratio=1.0):
        params_X = X_embedded.ravel()
        params_Y = Y_embedded.ravel()

        opt_args = {
            "it": 0,
            "n_iter_check": self._N_ITER_CHECK,
            "min_grad_norm": self.min_grad_norm,
            "learning_rate": self.learning_rate,
            "verbose": self.verbose,
            "kwargs": dict(skip_num_points=skip_num_points),
            "args": [P_X,P_Y, degrees_of_freedom, n_samples, self.n_components],
            "n_iter_without_progress": self._EXPLORATION_N_ITER,
            "n_iter": self._EXPLORATION_N_ITER,
            "momentum": 0.5,
        }

        opt_args['kwargs']['alpha'] = alpha
        opt_args['kwargs']['fixed_Y'] = fixed_Y
        opt_args['kwargs']['y']=y
        opt_args['kwargs']['oneplot']=oneplot
        opt_args['kwargs']['ratio']=ratio
        if self.method == 'barnes_hut':
            obj_func = _kl_divergence_yoke_bh
            opt_args['kwargs']['angle'] = self.angle
            opt_args['kwargs']['verbose'] = self.verbose
        else:
            obj_func = _kl_divergence_yoke

        P_X *= self.early_exaggeration
        if not fixed_Y:
            P_Y *= self.early_exaggeration

        params_X,params_Y, error,kl_x,kl_y, it = _gradient_descent_yoked(obj_func, params_X,params_Y,
                                                      **opt_args)
        if self.verbose:
            print("[t-SNE] KL divergence after %d iterations with early "
                  "exaggeration: %f" % (it + 1, kl_divergence))

        P_X /= self.early_exaggeration
        if not fixed_Y:
            P_Y /= self.early_exaggeration
        remaining = self.n_iter - self._EXPLORATION_N_ITER
        if it < self._EXPLORATION_N_ITER or remaining > 0:
            opt_args['n_iter'] = self.n_iter
            opt_args['it'] = it + 1
            opt_args['momentum'] = 0.8
            opt_args['n_iter_without_progress'] = self.n_iter_without_progress
            params_X,params_Y, error,kl_x,kl_y, it = _gradient_descent_yoked(obj_func, params_X,params_Y,
                                                      **opt_args)

        self.n_iter_ = it

        if self.verbose:
            print("[t-SNE] KL divergence after %d iterations: %f"
                  % (it + 1, kl_divergence))

        X_embedded = params_X.reshape(n_samples, self.n_components)
        Y_embedded = params_Y.reshape(n_samples, self.n_components)
        self.kl_divergencex = kl_x
        self.kl_divergencey = kl_y
        self.error = error

        return X_embedded,Y_embedded



    def fit_transform(self, X, y=None):
        embedding = self._fit(X)
        self.embedding_ = embedding
        return self.embedding_

    def fit(self, X, y=None):
        self.fit_transform(X)
        return self

    def Yoke_transform(self, X,Y,alpha,y=None, fixed_Y=False,oneplot=False,ratio=1.0):
        embedding_X,embedding_Y = self._yoke(X,Y,alpha,y=y,fixed_Y = fixed_Y,oneplot=oneplot,ratio=ratio)
        self.embedding_X = embedding_X
        self.embedding_Y = embedding_Y
        return self.embedding_X,self.embedding_Y

    def Yoke(self, X,Y, y=None,fixed_Y=False,oneplot=False,ratio=1.0):
        self._yoke(X,Y,alpha,y=y,fixed_Y = fixed_Y)
        return self

In [ ]:
alpha = 8

x_train = df[df["min"]==0]
z_train = df_new[df_new["min"]==0]
z_train = z_train.head(1000)
x_train = x_train.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_train = x_train.head(1000)
x_train = x_train.to_numpy()

train_result = tsne.fit_transform(x_train)

dff0 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff0 = pd.concat([dff0.reset_index(drop=True),z_train.reset_index(drop=True)],axis=1)

x_test = df[df["min"]==5]
z_test = df_new[df_new["min"]==5]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff1 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff1 = pd.concat([dff1.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==10]
z_test = df_new[df_new["min"]==10]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff2 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff2 = pd.concat([dff2.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==15]
z_test = df_new[df_new["min"]==15]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff3 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff3 = pd.concat([dff3.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==20]
z_test = df_new[df_new["min"]==20]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff4 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff4 = pd.concat([dff4.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==25]
z_test = df_new[df_new["min"]==25]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff5 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff5 = pd.concat([dff5.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==30]
z_test = df_new[df_new["min"]==30]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff6 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff6 = pd.concat([dff6.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==35]
z_test = df_new[df_new["min"]==35]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff7 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff7 = pd.concat([dff7.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==40]
z_test = df_new[df_new["min"]==40]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff8 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff8 = pd.concat([dff8.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==45]
z_test = df_new[df_new["min"]==45]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff9 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff9 = pd.concat([dff9.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==50]
z_test = df_new[df_new["min"]==50]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff10 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff10 = pd.concat([dff10.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


x_test = df[df["min"]==55]
z_test = df_new[df_new["min"]==55]
z_test = z_test.head(1000)
x_test = x_test.drop(['ts','DEST_NE_NAME','min'],axis=1)
x_test = x_test.head(1000)
x_test = x_test.to_numpy()

result,_= Yoked_TSNE(n_components=2,init_Y=train_result).Yoke_transform(x_test,0,alpha=10**-alpha,fixed_Y=True)

dff11 = pd.DataFrame(result, columns = ['comp_1','comp_2'])
dff11 = pd.concat([dff11.reset_index(drop=True),z_test.reset_index(drop=True)],axis=1)


df_final3 = pd.concat([dff0, dff1, dff2, dff3, dff4, dff5, dff6, dff7, dff8, dff9, dff10, dff11],axis=0)

df_final3.sort_values(by="min",ascending=True,inplace=True)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.025s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.000068
[t-SNE] KL divergence after 250 iterations with early exaggeration: 49.105766
[t-SNE] KL divergence after 1000 iterations: 0.241246


# **Performance Metrics for determining the quality of results**

In [ ]:
def measure_time(func):
    @wraps(func)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        elapsed = perf_counter() - start
        #print(f"{func.__qualname__.split('.')[0]} calculation "
        #      f"took {elapsed:.2f} seconds.")
        return result
    return inner


class Metric(ABC):

    @abstractmethod
    def _get_labels(self) -> list:
        pass

    @abstractmethod
    def calculate(self, df_data, df_labels) -> np.float32:
        pass

## **Shepard diagram**
### A Shepard diagram compares how far apart your data points are before and after you transform them (ie: goodness-of-fit) as a scatter plot. It is a scatterplot of distances between data points. On the x-axis, we plot the original distances. On the y-axis, we plot the distances output by a dimension reduction algorithm. Ideally it should be close to diagonal.

In [ ]:
class ShepardDiagram(Metric):
    def __init__(self, df_data: pd.DataFrame, df_embedding: pd.DataFrame, df_labels: pd.Series, n_sample: int = 30):
        self._df_data = df_data
        self._df_embedding = df_embedding
        self._df_labels = df_labels
        self.original_distances = None
        self.embedding_distances = None
        self._n_sample = n_sample

    def _get_labels(self) -> list:
        return list(set(self._df_labels))

    def _get_sample(self):
        indexes = []
        labels = self._get_labels()
        for label in labels:
            label_sample = self._df_labels[self._df_labels == label]
            indexes = indexes + list(label_sample)
        return indexes

    @staticmethod
    def _delete_diagonal(matrix):
        m = matrix.shape[0]
        strided = np.lib.stride_tricks.as_strided
        s0, s1 = matrix.strides
        return strided(matrix.ravel()[1:], shape=(m - 1, m), strides=(s0 + s1, s1)).reshape(m, -1)

    @measure_time
    def calculate(self):
        indexes = self._get_sample()
        original_sample = self._df_data.iloc[indexes]
        embedded_sample = self._df_embedding.iloc[indexes]

        original_distances = euclidean_distances(original_sample)
        original_distances = self._delete_diagonal(original_distances)
        embedded_distances = euclidean_distances(embedded_sample)
        embedded_distances = self._delete_diagonal(embedded_distances)

        self.original_distances = np.reshape(original_distances,
                                             (original_distances.shape[0] * original_distances.shape[1], 1))
        self.embedding_distances = np.reshape(embedded_distances,
                                              (embedded_distances.shape[0] * embedded_distances.shape[1], 1))

    def show(self):
        plt.scatter(self.original_distances, self.embedding_distances, alpha=0.7)
        plt.xlabel('Input distance')
        plt.ylabel('Output distance')
        plt.grid()
        plt.show()

## **Trustworthiness**
### It indicates to what extent the local structure is retained. The trustworthiness is within [0, 1].


In [ ]:
from sklearn.manifold import trustworthiness

class TrustworthinessBasedMetric(Metric):
    def __init__(self, df_data: pd.DataFrame, df_embedding: pd.DataFrame):
        self._df_data = df_data
        self._df_embedding = df_embedding
        self._n_metrics = ['euclidean', 'cosine']
        self._n_range = [5, 10, 15, 30, 50, 100, 150, 300, 500]

    def _get_labels(self):
        ...

    @measure_time
    def calculate(self):
        t=trustworthiness(
                    self._df_data, self._df_embedding, metric="euclidean"
                )
        return trustworthiness(self._df_data, self._df_embedding, metric="euclidean")


## **Continuity**
### It shows what fraction of neighbours are missing. The higher the rank (more distant) of the point, the greater the penalty.


In [ ]:
def continuity(D_high, D_low, k):
    n = D_high.shape[0]

    nn_orig = D_high.argsort()
    nn_proj = D_low.argsort()

    knn_orig = nn_orig[:, :k + 1][:, 1:]
    knn_proj = nn_proj[:, :k + 1][:, 1:]

    sum_i = 0

    for i in range(n):
        V = np.setdiff1d(knn_proj[i], knn_orig[i])

        sum_j = 0
        for j in range(V.shape[0]):
            sum_j += np.where(nn_proj[i] == V[j])[0] - k

        sum_i += sum_j

    return float((1 - (2 / (n * k * (2 * n - 3 * k - 1)) * sum_i)))

## **Normalized Stress**

In [ ]:
def normalized_stress(D_high, D_low):
    return (-1) * ((np.sum(D_high) - np.sum(D_low))**2 / np.sum(D_high**2) / 100)

# **Interactive Plot of the Time Series Dataset**

In [ ]:
def f(Choose_plot, Feature, Time):
  info = f"Visualization of Time Series dataset using {Choose_plot}."
  print(info)

  if Choose_plot == 't-SNE':
    data = df_final1
  elif Choose_plot == 'PCA':
    data = df_final4
  elif Choose_plot == 'UMAP':
    data = df_final5
  elif Choose_plot == 'Open t-SNE':
    data = df_final2
  elif Choose_plot == 'Joint t-SNE':
    data = df_final3

  original = df[df["min"] == Time]
  original = original.drop(['ts','DEST_NE_NAME', 'min'],axis=1)

  data = data[data["min"] == Time]
  data1 = data[['comp_1','comp_2']]
  y_km = km.fit_predict(data[['comp_1','comp_2']])

  if Feature == 'None':
    fig, ax = plt.subplots(figsize=(10, 10))
    plt.scatter(data['comp_1'], data['comp_2'], c=y_km ,s=50, cmap=plt.cm.Paired, alpha=0.7)
    plt.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],s=250, marker='*', label='centroids', edgecolor='black', c=np.arange(0,10), cmap=plt.cm.Paired,)
    fig.show()

  else:
    fig, ax = plt.subplots(figsize=(10, 10))
    sc=plt.scatter(data['comp_1'], data['comp_2'], c=data[Feature] ,s=50, cmap='Wistia', alpha=1)
    ax.legend(*sc.legend_elements(), title='clusters')
    plt.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],s=250, marker='*', label='centroids', edgecolor='black', c=np.arange(0,10), cmap=plt.cm.Paired,)
    fig.show()

  info = f"DR quality metrics for the plot are as under:"
  print(info)

  data_np = data1.to_numpy()

  tw=TrustworthinessBasedMetric(original,data1)
  print('Trustworthiness: ',tw.calculate())
  original1 = original.to_numpy()
  print('Continuity: ',continuity(original1,data_np, k=30))
  print('Normalized Stress: ',normalized_stress(original1,data_np))

  fig, ax = plt.subplots(figsize=(5, 5))
  sd=ShepardDiagram(original,data1,y_km,30)
  sd.calculate()
  sd.show()

In [ ]:
properties=['None', 'DNS Failure Latency (msec)', 'DNS Success Latency (msec)', 'Internet Latency (msec)', 'Round Trip Time (msec)', 'HTTP Latency (msec)', 'DL Data Volume (MB)', 'UL Data Volume (MB)']

drop = widgets.Dropdown(options=properties, value='None', description='Feature', disabled=False)

a = widgets.IntSlider(min=0, max=55, step=5, value=0)

r = widgets.ToggleButtons(
    options=['t-SNE', 'PCA', 'UMAP', 'Open t-SNE', 'Joint t-SNE'],
    description='DR Technique',
    disabled=False,
    button_style='',
)

interact(f , Choose_plot = r, Feature = drop, Time = a)


interactive(children=(ToggleButtons(description='DR Technique', options=('t-SNE', 'PCA', 'UMAP', 'Open t-SNE',…

<function __main__.f(Choose_plot, Feature, Time)>